# Lesson 4

## Training a neural network model

https://course.spacy.io/chapter4

In [2]:
import json

import spacy
from spacy.lang.en import English
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher

from spacy.tokens import Doc, Span, Token

# 
from print_util import print_doc_analysis, print_matcher_results

In [3]:
spacy.prefer_gpu()
nlp = English()
nlp = spacy.load("en_core_web_sm")

### 1 Training and Updating the Models

### 3 Creating Training Data 

#### Part 1

In [4]:
nlp = English()
matcher = Matcher(nlp.vocab)

In [5]:
with open("iphone.json") as f:
    TEXTS = json.loads(f.read())

In [8]:
pattern1 = [{"LOWER": "iphone"}, {"LOWER": "x"}]
pattern2 = [{"LOWER": "iphone"}, {"IS_DIGIT": True, "OP": "?"}]

In [9]:
matcher.add("GADGET", None, pattern1, pattern2)

In [10]:
TRAINING_DATA = []

In [14]:
for doc in nlp.pipe(TEXTS):
    # Match on the doc create list of matched spans
    spans = [doc[start:end] for match_id, start, end in matcher(doc)]
    # Get:  start char, end char, label tuples from each match
    entities = [(span.start_char, span.end_char, "GADGET") for span in spans]
    # Format the matcheas as a (doc.text, entities) tuple
    training_example = (doc.text, {"entities": entities})
    # Append the example to the training data
    TRAINING_DATA.append(training_example)

In [17]:
print (*TRAINING_DATA, sep="\n")

('How to preorder the iPhone X', {'entities': [(20, 28, 'GADGET'), (20, 26, 'GADGET')]})
('iPhone X is coming', {'entities': [(0, 8, 'GADGET'), (0, 6, 'GADGET')]})
('Should I pay $1,000 for the iPhone X?', {'entities': [(28, 36, 'GADGET'), (28, 34, 'GADGET')]})
('The iPhone 8 reviews are here', {'entities': [(4, 12, 'GADGET')]})
('Your iPhone goes up to 11 today', {'entities': [(5, 11, 'GADGET')]})
('I need a new phone! Any tips?', {'entities': []})
